In [1]:
# c.IPKernelApp.no_stderr = False

In [2]:
using Gadfly, DataFrames, DataArrays, Cairo, Images, ImageMagick, MultivariateStats, Colors;

    .==(Images.AbstractImageDirect{Bool, N<:Any}, AbstractArray{Bool, N<:Any}) at /Users/cwestrup/.julia/v0.4/Images/src/algorithms.jl:135
is ambiguous with: 
    .==(AbstractArray{Bool, N<:Any}, Union{DataArrays.PooledDataArray{Bool, R<:Integer, N<:Any}, DataArrays.DataArray{Bool, N<:Any}}) at /Users/cwestrup/.julia/v0.4/DataArrays/src/broadcast.jl:330.
To fix, define 
    .==(Images.AbstractImageDirect{Bool, N<:Any}, Union{DataArrays.PooledDataArray{Bool, R<:Integer, N<:Any}, DataArrays.DataArray{Bool, N<:Any}})
before the new definition.
    .==(Images.AbstractImageDirect, AbstractArray) at /Users/cwestrup/.julia/v0.4/Images/src/algorithms.jl:136
is ambiguous with: 
    .==(AbstractArray{Bool, N<:Any}, Union{DataArrays.PooledDataArray{Bool, R<:Integer, N<:Any}, DataArrays.DataArray{Bool, N<:Any}}) at /Users/cwestrup/.julia/v0.4/DataArrays/src/broadcast.jl:330.
To fix, define 
    .==(Images.AbstractImageDirect{Bool, N<:Any}, Union{DataArrays.PooledDataArray{Bool, R<:Integer, N<:Any}

In [3]:
set_default_plot_size(25cm, 20cm);

# Prerequisites

## Functions for evaluation

We'll define a few functions here to compute the mean F1 score

$$F1 = 2\frac{p * r}{p + r} \quad \text{where } p = \frac{tp}{tp + fp} \text{, }\quad r = \frac{tp}{tp + fn}$$

In [68]:
truePositives(y_true, y_pred, label) = sum(
    (y_pred .== label) & (y_true .== label));

In [69]:
falsePositives(y_true, y_pred, label) = sum(
    (y_pred .== label) & !(y_true .== label));

In [70]:
falseNegatives(y_true, y_pred, label) = sum(
    !(y_pred .== label) & (y_true .== label));

In [73]:
function precision(y_true, y_pred, label)
    tp = truePositives(y_true, y_pred, label)
    fp = falsePositives(y_true, y_pred, label)
    return tp / (tp + fp)
end;

In [72]:
function recall(y_true, y_pred, label)
    tp = truePositives(y_true, y_pred, label)
    fn = falseNegatives(y_true, y_pred, label)
    return tp / (tp + fn)
end;

In [74]:
function meanF1Score(y_true, y_pred, label)
    classes = sort(unique([unique(y_true);unique(y_pred)]))
    f1 = 0
    for class in classes
        p = precision(y_true, y_pred, label)
        r = recall(y_true, y_pred, label)
        f1 += 2 * ((p * r)/(p + r))        
    end
    return f1 / length(classes)
end;

Also we could use a function for the confusion matrix

In [10]:
function confusionMatrix(y_true, y_pred)
    classes = sort(unique([unique(y_true),unique(y_pred)]))
    confusion_matrix = zeros(Int64,length(classes),length(classes))
    for i in 1:length(y_true)
        # translate label into index
        true_class = findfirst(classes,y_test[i])
        pred_class = findfirst(classes,y_pred[i])
        # pred class is the row, true class is the column
        cm[pred_class,true_class] += 1
    end
    return confusion_matrix
end;

## Functions for cross-validation

### Generating k folds by splitting the data in test and training set

A function to generate vectors for splitting the dataset for cross validation each vector selects the datapoints for the test set.

In [11]:
# data_size: data points / rows in dataset
# folds: number of split vectors to be used for CV folds
# test_size_ratio: percentage of dataset to be in test set
function getCVSplits(data_size=5000, folds=5, test_size_ratio=0.25)
    splits = []
    for k in range(1,folds)
        is_train = shuffle(
            range(1, data_size) .> floor(n * test_size_ratio))
        push!(splits, is_train)
    end
    return splits
end;

A function that splits our dataset with a boolean vector into test and training set:

In [12]:
# splitvectors: boolean vector that selects which entries are in 
#              test/training set, can be obtained using getCVSplits()
# X: feature matrix DataFrame
# Y: target matrix DataFrame
function splitDataset(splitvector, X, Y)
    X_train_df = X[splitvector,:]
    X_test_df  = X[!splitvector,:]
    Y_train_df = Y[splitvector, :]
    Y_test_df  = Y[!splitvector, :]
    return (X_train_df, X_test_df, Y_train_df, Y_test_df)
end;

### Generating feature matrix and target vectors from split data

A function to get the feature matrix with added bias

In [13]:
getFeatureMatrix(Xdf) = hcat(ones(nrow(Xdf), 1), Matrix(Xdf));

A funtion to get the target vector for predicting the type of wine (classification task)

In [14]:
# assign class labels -1 for Red and 1 for White
function getTargetTypeVector(Ydf)
    Y = ones(nrow(Ydf))
    is_white = collect(Matrix(Ydf[:,2:2]) .== "White")
    Y[!is_white,:] = -1.0;
    return Y
end;

A funtion to get the target vector for predicting the quality of wine (regression task)

In [15]:
getTargetQualityVector(Ydf) = Matrix(Ydf[:,1:1]);

## Data preparation

We can define the feature variables we'll use:

In [16]:
variables = [:fixedAcidity, :volatileAcidity, :citricAcid, 
    :residualSugar, :chlorides, :freeSulfurDioxide, 
    :totalSulfurDioxide, :density, :pH, :sulphates, :alcohol];
length(variables)

11

Read the data into DataFrames:

In [17]:
df_submission = 
readtable("data/challenge_public_test_classification_regression_2015.csv");

In [18]:
df_train = readtable("data/training_classification_regression_2015.csv");

We normalize the features by subtracting from each column its mean and deviding by its standard deviation

In [19]:
df_train_norm = deepcopy(df_train)
for variable in variables
    column = df_train_norm[:, variable]
    df_train_norm[:, variable] = (column .- mean(column)) ./ std(column)
end
df_train_norm;

In [20]:
head(df_train_norm, 1)

,fixedAcidity,volatileAcidity,citricAcid,residualSugar,chlorides,freeSulfurDioxide,totalSulfurDioxide,density,pH,sulphates,alcohol,quality,_type
1,-0.19581148606307708,-0.3386149588803073,0.5430063966089778,1.6899239900642382,-0.12119819492048489,1.9275660835074595,2.2540949093649036,1.0116150901531629,-0.3105996734319634,0.38998483249693666,-0.8034568236260613,4,White


Stack up the features for easier visualization

In [21]:
df_train_norm_stacked = stack(df_train_norm, variables);

In [22]:
head(df_train_norm_stacked, 1)

,variable,value,quality,_type
1,fixedAcidity,-0.19581148606307708,4,White



Preparing the data for test / training:

In [23]:
# X = feature DataFrame, Y = target DataFrame
X_df = df_train_norm[:,1:11]
Y_df = df_train_norm[:,12:13];

## Generating the test and training set

Let's try it out and split the dataset with 0.75 training and 0.25 test data:

In [24]:
# Split for training and test set
n = nrow(X_df)
splits = getCVSplits(n, 1, 0.25)

# Create dataframes with training and test sets:
(X_train_df, X_test_df, Y_train_df, Y_test_df) = splitDataset(
    splits[1], X_df, Y_df)

println("Total number of datapoints: ",n)
println("Training set size: ", nrow(X_train_df))
println("Test set size: ", nrow(X_test_df))

Total number of datapoints: 5000
Training set size: 3750
Test set size: 1250


Now we can generate the feature matrix and target vectors for the classification and regression tasks:

In [25]:
X_train = getFeatureMatrix(X_train_df);
X_test = getFeatureMatrix(X_test_df);

y_type_train = getTargetTypeVector(Y_train_df);
y_type_test = getTargetTypeVector(Y_test_df);
y_quality_train = getTargetQualityVector(Y_train_df);
y_quality_test = getTargetQualityVector(Y_test_df);

# Visualizations

## Classification

A placeholder plot we'll use to fill up grids

In [ ]:
placeholder = plot(x=[0], y=[0], Guide.xlabel("'placeholder'"));

Plotting all classes for all variables

In [ ]:
filename = "hist_variables_type"
p = plot(df_train_norm_stacked,
xgroup="variable", x="_type", y="value", Geom.subplot_grid(Geom.point));
draw(PDF(string("plots/", filename, ".pdf"), 25cm, 20cm), p)
draw(PNG(string("plots/", filename, ".png"), 25cm, 20cm), p)
load(string("plots/", filename, ".png"))

Now each seperately 

In [ ]:
plots = Gadfly.Plot[]

for variable in variables
    feature = df_train_norm_stacked[
        (df_train_norm_stacked[:variable].==variable), :] 
    push!(plots, plot(feature, x=:_type, y=:value, Geom.point, 
        Guide.xlabel(string(variable), orientation=:horizontal)))
end
push!(plots, placeholder);

In [ ]:
filename = "hist_variables_type_grid"

plotstack = vstack(
    hstack(plots[1], plots[2], plots[3], plots[4]),
    hstack(plots[5], plots[6], plots[7], plots[8]),
    hstack(plots[9], plots[10], plots[11], plots[12]))

draw(PDF(string("plots/", filename, ".pdf"), 25cm, 20cm), plotstack)
draw(PNG(string("plots/", filename, ".png"), 25cm, 20cm), plotstack)
load(string("plots/", filename, ".png"))

The features look somewhat seperable, let's look at the density

In [ ]:
plots = Gadfly.Plot[]

for variable in variables
    feature = df_train_norm_stacked[
        (df_train_norm_stacked[:variable].==variable), :] 
    push!(plots, plot(feature, color=:_type, x=:value, Geom.density,
    Guide.xlabel(string(variable), orientation=:horizontal), 
    Coord.cartesian(xmin=-5, xmax=5)))
end
push!(plots, placeholder);

In [ ]:
filename = "density_type_grid"
plotstack = vstack(
    hstack(plots[1], plots[2]),
    hstack(plots[3], plots[4]),
    hstack(plots[5], plots[6]),
    hstack(plots[7], plots[8]),
    hstack(plots[9], plots[10]),
    hstack(plots[11], plots[12]))
draw(PDF(string("plots/", filename, ".pdf"), 25cm, 28cm), plotstack)
draw(PNG(string("plots/", filename, ".png"), 25cm, 28cm), plotstack)
load(string("plots/", filename, ".png"))

Seems like there is some pattern here at least...

In [ ]:
plots = Gadfly.Plot[]

for variable in variables
    feature = df_train_norm_stacked[
        (df_train_norm_stacked[:variable].==variable), :] 
    
    push!(plots, plot(feature, color=:quality, x=:value, Geom.density,
    Guide.xlabel(string(variable), orientation=:horizontal), 
    Scale.ContinuousColorScale(
        Scale.lab_gradient(colorant"orange", colorant"blue")),
    Coord.cartesian(xmin=-5, xmax=5)))
end
push!(plots, placeholder);

In [ ]:
filename = "density_quality_grid"
plotstack = vstack(
    hstack(plots[1], plots[2]),
    hstack(plots[3], plots[4]),
    hstack(plots[5], plots[6]),
    hstack(plots[7], plots[8]),
    hstack(plots[9], plots[10]),
    hstack(plots[11], plots[12]))
draw(PDF(string("plots/", filename, ".pdf"), 25cm, 28cm), plotstack)
draw(PNG(string("plots/", filename, ".png"), 25cm, 28cm), plotstack)
imread(string("plots/", filename, ".png"))

## PCA

In [ ]:
# train a PCA model
X_matrix = Matrix(deepcopy(X_df))
M = fit(PCA, X_matrix'; maxoutdim=15, method=:auto)
pca_matrix = transform(M, X_matrix')';

In [ ]:
# make it a DataFrame and merge it with the labels
df_pca = DataFrame(pca_matrix)
df_pca_labelled = hcat(df_pca, Y_df)
head(df_pca_labelled,1)

Plot first 2 principal components:

In [ ]:
filename = "density_pca_type_2first"
p1 = plot(df_pca_labelled, color=:_type, x=:x1, Geom.density)
p2 = plot(df_pca_labelled, color=:_type, x=:x2, Geom.density)
plotstack = hstack(p1, p2)

draw(PDF(string("plots/", filename, ".pdf"), 25cm, 10cm), plotstack)
draw(PNG(string("plots/", filename, ".png"), 25cm, 10cm), plotstack)
load(string("plots/", filename, ".png"))

First 2 components on a plane

In [ ]:
filename = "scatter_pca_type_2first"
p = plot(df_pca_labelled, color=:_type, x=:x1, y=:x2, Geom.point)
draw(PDF(string("plots/", filename, ".pdf"), 20cm, 20cm), p)
draw(PNG(string("plots/", filename, ".png"), 20cm, 20cm), p)
load(string("plots/", filename, ".png"))

In [ ]:
filename = "scatter_pca_quality_2first"
p = plot(df_pca_labelled, color=:quality, x=:x1, y=:x2, Geom.point, 
    Scale.ContinuousColorScale(
    Scale.lab_gradient(colorant"orange", colorant"blue")))
draw(PDF(string("plots/", filename, ".pdf"), 20cm, 20cm), p)
draw(PNG(string("plots/", filename, ".png"), 20cm, 20cm), p)
load(string("plots/", filename, ".png"))

In [ ]:
plots = Gadfly.Plot[]

variables = [:x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10]
df_pca_labelled_stacked = stack(df_pca_labelled, variables)

for variable in variables
    feature = df_pca_labelled_stacked[
        (df_pca_labelled_stacked[:variable].==variable), :]     
    push!(plots, plot(feature, color=:quality, x=:value, Geom.density,
    Guide.xlabel(string(variable), orientation=:horizontal), 
    Scale.ContinuousColorScale(
        Scale.lab_gradient(colorant"orange", colorant"blue")),
    Coord.cartesian(xmin=-5, xmax=5)))
end

In [ ]:
filename = "density_pca_quality_grid"
plotstack = vstack(
    hstack(plots[1], plots[2]),
    hstack(plots[3], plots[4]),
    hstack(plots[5], plots[6]),
    hstack(plots[7], plots[7]),
    hstack(plots[8], plots[9]))
draw(PDF(string("plots/", filename, ".pdf"), 25cm, 25cm), plotstack)
draw(PNG(string("plots/", filename, ".png"), 25cm, 25cm), plotstack)
imread(string("plots/", filename, ".png"))

In [ ]:
plots = Gadfly.Plot[]

variables = [:x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10]
df_pca_labelled_stacked = stack(df_pca_labelled, variables)

for variable in variables
    feature = df_pca_labelled_stacked[
        (df_pca_labelled_stacked[:variable].==variable), :]     
    push!(plots, plot(feature, color=:_type, x=:value, Geom.density,
    Guide.xlabel(string(variable), orientation=:horizontal), 
    Coord.cartesian(xmin=-5, xmax=5)))
end

In [ ]:
filename = "density_pca_type_grid"
plotstack = vstack(
    hstack(plots[1], plots[2]),
    hstack(plots[3], plots[4]),
    hstack(plots[5], plots[6]),
    hstack(plots[7], plots[7]),
    hstack(plots[8], plots[9]))
draw(PDF(string("plots/", filename, ".pdf"), 25cm, 25cm), plotstack)
draw(PNG(string("plots/", filename, ".png"), 25cm, 25cm), plotstack)
imread(string("plots/", filename, ".png"))

# ICA

In [ ]:
# train a ICA model
X_matrix = Matrix(deepcopy(X_df))
M = fit(ICA, X_matrix', 6)
ica_matrix = transform(M, X_matrix')';

In [ ]:
df_ica = DataFrame(ica_matrix)
labels = df_train[:,12:13]
df_ica_labelled = hcat(df_ica, labels);

In [ ]:
plots = Gadfly.Plot[]

variables = [:x1, :x2, :x3, :x4, :x5, :x6]
for variable in variables
    push!(plots, plot(df_ica_labelled, color=:_type, 
        x=variable, Geom.density,
        Guide.xlabel(string(variable), orientation=:horizontal),
        Coord.cartesian(xmin=-5, xmax=5)))
end

In [ ]:
filename = "density_ica_type_grid"
plotstack = vstack(
    hstack(plots[1], plots[2]),
    hstack(plots[3], plots[4]),
    hstack(plots[5], plots[6]))

draw(PDF(string("plots/", filename, ".pdf"), 25cm, 25cm), plotstack)
draw(PNG(string("plots/", filename, ".png"), 25cm, 25cm), plotstack)
imread(string("plots/", filename, ".png"))

In [ ]:
plots = Gadfly.Plot[]

variables = [:x1, :x2, :x3, :x4, :x5, :x6]
for variable in variables
    push!(plots, plot(df_ica_labelled, color=:quality, 
        x=variable, Geom.density,
        Guide.xlabel(string(variable), orientation=:horizontal),
        Scale.ContinuousColorScale(
            Scale.lab_gradient(colorant"orange", colorant"blue")),
        Coord.cartesian(xmin=-5, xmax=5)))
end

In [ ]:
filename = "density_ica_quality_grid"
plotstack = vstack(
    hstack(plots[1], plots[2]),
    hstack(plots[3], plots[4]),
    hstack(plots[5], plots[6]))

draw(PDF(string("plots/", filename, ".pdf"), 25cm, 25cm), plotstack)
draw(PNG(string("plots/", filename, ".png"), 25cm, 25cm), plotstack)
imread(string("plots/", filename, ".png"))

# Linear regression

### Description 

(*Add reference to Bishop)

Train the weights throught the closed-form solution using the pseudo-inverse matrix of $X$

$w(X,Y) = (X^Y X)^{-1} X^Y = X^\dagger Y$

In [26]:
wLinearRegression(X, Y) = pinv(X)*Y;

We can then make predictions using these weights:

$y(X, w) = Xw $

In [27]:
yLinearRegression(X, w) = X*w;

### Test

Train our weights:

In [28]:
w = wLinearRegression(X_train, y_type_train);

Make predictions:

In [29]:
y_pred = sign(yLinearRegression(X_test, w));

F1 score:

In [77]:
meanF1Score(y_pred, y_type_test, -1.0)

0.9845474613686535

# Logistic regression

Let's implement logistic regression from scratch to start simple:

$g(x) =  \sum_{i=0}^k w_i Φ_i(x)$, with $Φ_0(x) = 0$

# Neural Network

### Backpropagation:

Number of features is the size of the training set columns, without the id, type and quality columns

In [ ]:
num_of_features = size(df_train,2)-3

Specify the architechture. We'll have a fully connected MLP with 1 hidden layer.

In [ ]:
hidden_layers = 1;
hidden_units = 20;
num_of_outputs = 1;

Initialize the weight matrices randomly

In [ ]:
using Distributions 
θ1 = rand(Uniform(-1, 1), hidden_units + 1, num_of_features + 1);
θ1[1, :] = 1;
θ1

In [ ]:
θ2 = rand(Uniform(-1, 1), num_of_outputs, hidden_units + 1);

In [ ]:
size(θ1), size(θ2)

In [ ]:
x = rand(10)

In [ ]:
x = [1;x]

In [ ]:
tanh(θ1 * x)

In [ ]:
function forwardprop(x,θ)
    tanh(θ ⋅ x)
end